In [1]:
!nvidia-smi

Thu Aug  8 21:19:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               Off | 00000000:01:00.0 Off |                  Off |
| 58%   82C    P2             299W / 300W |   9489MiB / 49140MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!gpustat --debug


cil-hydra            Thu Aug  8 21:20:07 2024  535.183.01
[0] NVIDIA RTX A6000 | 82°C, 100 % |  9489 / 49140 MB | ongzhiyang(4740M) ongzhiyang(4740M)
[1] NVIDIA RTX A6000 | 79°C,  99 % | 10758 / 49140 MB | ongzhiyang(3582M) ongzhiyang(3582M) ongzhiyang(3582M)
[2] NVIDIA RTX A6000 | 72°C,  56 % |  4746 / 49140 MB | ongzhiyang(4740M)
[3] NVIDIA RTX A6000 | 51°C,  32 % |  3588 / 49140 MB | ongzhiyang(3582M)


In [3]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Generate subset of training data

In [1]:
import random
random.seed(42) # Set a random seed for reproducibility

dataset_path = "/home/FYP/on0008an/bias-bench-main/data/wikipedia-10.txt"
sampling_ratio = 0.25
sampled_dataset_path = "/home/FYP/on0008an/bias-bench-main/data/wikipedia-10_sample.txt"

# Read
with open(dataset_path, "r") as f:
    lines = f.readlines()

sampled_lines = random.sample(lines, int(len(lines) * sampling_ratio))

# Write sampled dataset
with open(sampled_dataset_path, "w") as f:
    f.writelines(sampled_lines)

# Train models with CDA data

## All possible arguments:
- https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/trainer#transformers.TrainingArguments
- ModelArguments (In the script)
- DataTrainingArguments (In the script)

## Lora explanation

- Instead of updating weights, Lora tracks changes
- Changes are tracked in 2 smaller matrices, multiplied to form the same size matrices as the model original weights

Parameters:
- Rank: Number of columns in the 2 smaller matrices
    - As rank increases, more parameters are fine-tuned
    - Downstream tasks intrinsically work well with low rank
    - However, complex tasks/behavior that contradicts the pre-training dataset may require higher rank
- Training all layers of the network is essential to match full-training performance
- Alpha: Scaling factor that is applied to the weight changes when adding to original weights, scale factor = Alpha/Rank
    - Microsoft Lora paper sets Alpha = 2 * rank
    - QLora paper sets Alpha = 1/4 * rank
- Dropout: Randomly set a fraction of the weight changes to zero to prevent overfitting
    - QLora paper sets dropout = 0.1 for 7B, 13B models, 0.05 for 33B, 65B models

# To achieve the most out of Lora fine-tuning, we must train all layers of the network

Llama 2:
- Linear Layers: gate_proj, down_proj, up_proj, q_proj, v_proj, k_proj, and o_proj

#If only targeting attention blocks of the model
target_modules = ["q_proj", "v_proj"]

#If targeting all linear layers
target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']

# Following parameters from the batch script

- Cache takes around 3 runs to be built

## For GPT2

In [ ]:
!python run_clm.py --model_name_or_path "gpt2" --tokenizer_name "gpt2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 42 --output_dir "../results/CDA_FT/gpt2/gender" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora

/home/ongzhiyang/anaconda3/envs/fyp/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
07/07/2024 08:41:23 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
Padding token: <|endoftext|>
Quantization enabled
Training target modules: ['query', 'value']
Applying counterfactual augmentation:  25%|▏| 1322000/5349612 [34:58<1:50:10, 60

In [ ]:
!python run_clm.py --model_name_or_path "gpt2" --tokenizer_name "gpt2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "race" --seed 42 --output_dir "../results/CDA_FT/gpt2/race" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora

In [ ]:
!python run_clm.py --model_name_or_path "gpt2" --tokenizer_name "gpt2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "religion" --seed 42 --output_dir "../results/CDA_FT/gpt2/religion" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora

In [12]:
# 500 max_steps to fit into 6hours training timeframe
!python run_clm.py --model_name_or_path "gpt2" --do_train --train_file "../data/wikipedia-10_tiny.txt" --max_steps 500 --per_device_train_batch_size 8 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 0 --output_dir "../results/CDA_FT/gpt2" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024

/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
06/13/2024 23:20:04 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
06/13/2024 23:20:20 - WARNING - accelerate.utils.other - Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
[WARNING|trainer.py:

## To continue training from a checkpoint, need to set steps to the number of steps already done + what you want to do

In [1]:
# Continue from checkpoint
!python run_clm.py --model_name_or_path "gpt2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 1000 --per_device_train_batch_size 8 --gradient_accumulation_steps 32 --save_steps 500 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 0 --output_dir "../results/gpt2_CDA_training_seed0_1kstep" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --resume_from_checkpoint "../results/gpt2_CDA_training_seed0/checkpoint-500"

/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
06/12/2024 10:32:36 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
06/12/2024 10:32:37 - WARNING - datasets.builder - Using custom data configuration default-8f3a313a0598a372
06/12/2024 10:32:37 - WARNING - datasets.builder - Reusing dataset text (/home/FYP/on0008an/.cache/huggingface/datasets/text/default-8f3a313a0598a372/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 67.07it/s]
06/12/2024 10:32:38 - WARNING - datasets.builder - Using custom data configuration default-8f3a313a0598a372
06/12/2024 10:32:38 - WARNING - datasets.builder - Reusing datas

## Phi 2.0

## Gender

In [ ]:
!python run_clm.py --model_name_or_path "microsoft/phi-2" --tokenizer_name "microsoft/phi-2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 42 --output_dir "../results/CDA_FT/phi2/gender" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora

## Religion

In [ ]:
!python run_clm.py --model_name_or_path "microsoft/phi-2" --tokenizer_name "microsoft/phi-2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "religion" --seed 42 --output_dir "../results/CDA_FT/phi2/religion" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora

In [ ]:
!CUDA_VISIBLE_DEVICES=2 nohup python run_clm.py --model_name_or_path "microsoft/phi-2" --tokenizer_name "microsoft/phi-2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "religion" --seed 42 --output_dir "../results/CDA_FT/phi2/religion" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora &

## Race

In [ ]:
!CUDA_VISIBLE_DEVICES=2 nohup python run_clm.py --model_name_or_path "microsoft/phi-2" --tokenizer_name "microsoft/phi-2" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "race" --seed 42 --output_dir "../results/CDA_FT/phi2/race" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora &

## For llama 2, max block size/sequence length is 1024 (OOM error)

## Gender

In [ ]:
!python run_clm.py --model_name_or_path "meta-llama/Llama-2-7b-hf" --tokenizer_name "meta-llama/Llama-2-7b-hf" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 42 --output_dir "../results/CDA_FT/llama2/gender" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora

/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
06/22/2024 02:58:50 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Continue from checkpoint
!python run_clm.py --model_name_or_path "meta-llama/Llama-2-7b-hf" --tokenizer_name "meta-llama/Llama-2-7b-hf" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 42 --output_dir "../results/CDA_FT/llama2/gender" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora --resume_from_checkpoint "../results/CDA_FT/llama2/gender/checkpoint-900"

/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
06/27/2024 06:34:48 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Padding token: </s>
Quantization enabled
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:06<00:00,  3.45s/it]
/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argume

## Race

In [ ]:
!python run_clm.py --model_name_or_path "meta-llama/Llama-2-7b-hf" --tokenizer_name "meta-llama/Llama-2-7b-hf" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "race" --seed 42 --output_dir "../results/CDA_FT/llama2/race" --persistent_dir "/home/ongzhiyang/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora

In [ ]:
# Continue from checkpoint
!python run_clm.py --model_name_or_path "meta-llama/Llama-2-7b-hf" --tokenizer_name "meta-llama/Llama-2-7b-hf" --do_train --train_file "../data/wikipedia-10.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "gender" --seed 42 --output_dir "../results/CDA_FT/llama2/gender" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora --resume_from_checkpoint "../results/CDA_FT/llama2/gender/checkpoint-200"

/home/FYP/on0008an/.conda/envs/RunJupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
06/21/2024 15:44:16 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False


## Religion

In [ ]:
!python run_clm.py --model_name_or_path "meta-llama/Llama-2-7b-hf" --tokenizer_name "meta-llama/Llama-2-7b-hf" --do_train --train_file "../data/wikipedia-10_sample.txt" --max_steps 2000 --per_device_train_batch_size 1 --gradient_accumulation_steps 32 --save_steps 100 --preprocessing_num_workers 1 --counterfactual_augmentation "religion" --seed 42 --output_dir "../results/CDA_FT/llama2/religion" --persistent_dir "/home/FYP/on0008an/bias-bench-main" --num_train_epochs 3 --overwrite_output_dir --logging_strategy "steps" --logging_steps 1 --block_size 1024 --lora